## 03.클래스로 파이토치 모델 구현하기

파이토치의 대부분의 구현체들은 대부분 모델을 생성할 때 클래스(Class)를 사용하고 있다. 앞서 배운 선형 회귀를 클래스로 구현해보겠다. 앞서 구현한 코드와 다른 점은 오직 클래스로 모델을 구현했다는 점이다.

### 1.모델을 클래스로 구현하기

앞서 로지스틱 회귀 모델은 다음과 같이 구현했었다.

``` python
model = nn.Sequential(
   nn.Linear(2, 1), # input_dim = 2, output_dim = 1
   nn.Sigmoid() # 출력은 시그모이드 함수를 거친다
)
```

이를 클래스로 구현하면 다음과 같다.

``` python
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))
```

위와 같은 클래스를 사용한 모델 구현 형식은 대부분의 파이토치 구현체에서 사용하고 있는 방식으로 반드시 숙지할 필요가 있다.

클래스(class) 형태의 모델은 nn.Module을 상속받는다. 그리고 __init__()에서 모델의 구조와 동작을 정의하는 생성자를 정의한다. 이는 파이썬에서 객체가 갖는 속성값을 초기화하는 역할로, 객체가 생성될 때 자동으로 호출된다. super() 함수를 부르면 여기서 만든 클래스는 nn.Module 클래스의 속성들을 가지고 초기화 된다. forward() 함수는 모델이 학습데이터를 입력받아서 forward 연산을 진행시키는 함수이다. 이 forward() 함수는 model 객체를 데이터와 함께 호출하면 자동으로 실행이 된다. 예를 들어 model이란 이름의 객체를 생성 후, model(입력 데이터)와 같은 형식으로 객체를 호출하면 자동으로 forward 연산이 수행된다.

* $H(x)$ 식에 입력 $x$로부터 예측된 $y$를 얻는 것을 forward 연산이라고 한다.



### 2.로지스틱 회귀 클래스로 구현하기

이제 모델을 클래스로 구현한 코드를 보겠다. 달라진 점은 모델을 클래스로 구현했다는 점 뿐이다. 다른 코드는 전부 동일하다.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [3]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [4]:
class BinaryClassifier(nn.Module):

    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))
        

In [5]:
model = BinaryClassifier()
model

BinaryClassifier(
  (linear): Linear(in_features=2, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [8]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr = 1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 10번마다 로그 출력
    if epoch % 10 == 0:

        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산

        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(epoch, nb_epochs,
                                                                     cost.item(),
                                                                     accuracy * 100))

Epoch    0/1000 Cost: 0.620154 Accuracy 50.00%
Epoch   10/1000 Cost: 0.493699 Accuracy 66.67%
Epoch   20/1000 Cost: 0.426488 Accuracy 66.67%
Epoch   30/1000 Cost: 0.361094 Accuracy 83.33%
Epoch   40/1000 Cost: 0.305882 Accuracy 83.33%
Epoch   50/1000 Cost: 0.256603 Accuracy 83.33%
Epoch   60/1000 Cost: 0.212054 Accuracy 100.00%
Epoch   70/1000 Cost: 0.175886 Accuracy 100.00%
Epoch   80/1000 Cost: 0.154053 Accuracy 100.00%
Epoch   90/1000 Cost: 0.141991 Accuracy 100.00%
Epoch  100/1000 Cost: 0.132478 Accuracy 100.00%
Epoch  110/1000 Cost: 0.124198 Accuracy 100.00%
Epoch  120/1000 Cost: 0.116910 Accuracy 100.00%
Epoch  130/1000 Cost: 0.110446 Accuracy 100.00%
Epoch  140/1000 Cost: 0.104674 Accuracy 100.00%
Epoch  150/1000 Cost: 0.099488 Accuracy 100.00%
Epoch  160/1000 Cost: 0.094803 Accuracy 100.00%
Epoch  170/1000 Cost: 0.090550 Accuracy 100.00%
Epoch  180/1000 Cost: 0.086671 Accuracy 100.00%
Epoch  190/1000 Cost: 0.083119 Accuracy 100.00%
Epoch  200/1000 Cost: 0.079854 Accuracy 100.00